In [1]:
import math
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import time as t

In [2]:
import pandas as pd
import sys
from env_pybullet_gen3 import env_pybullet_kin_gen3



In [3]:
#To improve the velocity, run it on the GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device ', device)



#Create a experiment env
env = env_pybullet_kin_gen3(no_zeros = True,Excel_path_Okay_tcp = "./test_tcp_19_averageconverted.xlsx",time_step=0.005)
env.robot.visual_inspection = False

#Initially parameters of the urdf



print('observation space:', env.observation_space) #states, There is only 1 state constant
env.update_parameters_to_modify(["mass","max_vel","kp","ki","damping","force_x_one","Ixx","Iyy","Izz"])
print('action space:', env.action_space) #parameters, number of parameters choose to tune, continuous
print('original action:', env.action_original()) #parameters, number of parameters choose to tune, continuous




Device  cpu
hola
../Simulation_Pybullet/models/urdf/JACO3_URDF_V11.urdf
Robot launched
hola
(7, 12)
(7, 17)
observation space: 1
mass okey
max_vel okey
kp okey
ki okey
damping okey
force_x_one okey
Ixx okey
Iyy okey
Izz okey
action space: 63
original action: [1.377353, 1.163667, 1.16366, 0.930287, 0.678106, 0.678106, 0.500657, 30, 30, 30, 30, 30, 30, 30, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 1, 1, 1, 1, 1, 1, 1, 0.00480078220558528, 0.008418724123621643, 0.007544516197001279, 0.0064096919604697605, 0.0016797846804208455, 0.0019375935324615593, 0.0007750385804833523, 0.004755191268457921, 0.0019202057294098781, 0.007486605057526543, 0.0013804130332615912, 0.0015062421641084327, 0.0008273237988932355, 0.0005849825981943527, 0.0022826303695446856, 0.00836116845951151, 0.0019205500000651847, 0.006517816917001926, 0.0008260694053789821, 0.0017630597813546379, 0.0009751695712112207]


In [4]:
#Cross Entrophy Method, to choose the weights

# In my case where only 1 action,and that it's apply the parameters do another step doesn't change anything due to the state doesn't vary
# For this reason max_t and gama doesn't make sense, so I set them to max_t = 1 and gamma to 0
def cem_no_net(n_iterations=600, max_t=1, gamma=0.0, print_every=100, pop_size=env.action_space, elite_frac=0.1, sigma=0.05,sigma_reduction_every_print = 0.65, per_one = True ):
    """PyTorch implementation of the cross-entropy method.
        
    Params
    ======
        n_iterations (int): maximum number of training iterations
        max_t (int): maximum number of timesteps per episode
        gamma (float): discount rate
        print_every (int): how often to print average score (over last 100 episodes)
        pop_size (int): size of population at each iteration
        elite_frac (float): percentage of top performers to use in update
        sigma (float): standard deviation of additive noise
        per_one (boolean): to determine if the output is in per one or not
    """
    #Numbers of elements that you keep as the better ones
    n_elite=int(pop_size*elite_frac)
    
    #scores doble end queee , from iterations size * 0.1
    scores_deque = deque(maxlen=int(n_iterations*0.1))
    #intial scores empty
    scores = []
    #Select a seed to make the results the same every test, not depending on the seed
    np.random.seed(0)
    #Initial best weights, are from 0 to 1, it's good to be small the weights, but they should be different from 0.
    # small to avoid overfiting , different from 0 to update them
    
    if (per_one == True):
        best_weight = sigma*np.random.randn(env.action_space)
        original_action = np.array(env.action_original())
    else:
        best_weight = np.add(sigma*np.random.randn(env.action_space),env.action_original())

    #Each iteration, modify  + (from 0 to 1) the best weight randomly
    #Computes the reward with these weights
    #Sort the reward to get the best ones
    # Save the best weights
    # the Best weight it's the mean of the best one
    #compute the main reward of the main best rewards ones
    #this it's show to evalute how good its
    
    for i_iteration in range(1, n_iterations+1):
        
        #Generate new population weights, as a mutation of the best weight to test them
        weights_pop = [best_weight + (sigma*np.random.randn(env.action_space)) for i in range(pop_size)]
        
        #Compute the parameters and obtain the rewards for each of them
        #print("iteration "+str(i_iteration))
        if (per_one == True):
            rewards=[]
            for weights in weights_pop:
                #print("New weights")
                action=np.add(np.multiply(weights,original_action),original_action)
                #t.sleep(10)
                rewards.append( env.step_tcp_rishabh(action) )
            rewards = np.array(rewards)
        else:
            rewards=[]
            for weights in weights_pop:
                rewards.append(env.step_tcp_rishabh(weights) )
            rewards = np.array(rewards)
        print("rewards" + str(i_iteration))
        print(rewards)
        #print("\n")
        
        #Sort the rewards to obtain the best ones
        elite_idxs = rewards.argsort()[-n_elite:]
        elite_weights = [weights_pop[i] for i in elite_idxs]
        
        #Set the best weight as the mean of the best ones 
       
        best_weight = np.array(elite_weights).mean(axis=0)
        
        #Get the reward with this new weight
        
        if (per_one == True):
            action = np.add(np.multiply(best_weight,original_action),original_action)
            reward = env.step_tcp_rishabh(action)
            print(reward)
        else:
            reward = env.step_tcp_rishabh(best_weight)
        scores_deque.append(reward)
        scores.append(reward)
        
        #save the check point
        env.save_parameters("./Parameters_train_tcp_euc.xlsx")
        
        if i_iteration % print_every == 0:
            sigma = sigma * sigma_reduction_every_print
            print('Episode {}\tAverage Score: {:.2f}'.format(i_iteration, np.mean(scores_deque)))

        if np.mean(scores_deque)>=0.0:
            print('\nEnvironment solved in {:d} iterations!\tAverage Score: {:.2f}'.format(i_iteration-n_iterations*0.1, np.mean(scores_deque)))
            break
    return scores


In [ ]:
#Execute the cross entrophy method with default Values
#scores = cem()


#To don't ask the GPU as much reduce the pop_size, it's the amount of elemts try
scores = cem_no_net()
# 
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

rewards1
[-0.67096293 -0.67092371 -0.67083827 -0.67076827 -0.67068258 -0.67116003
 -0.6709193  -0.67066202 -0.67107155 -0.67101908 -0.67074551 -0.67084049
 -0.6709379  -0.67108836 -0.67092862 -0.67101246 -0.6709995  -0.67095842
 -0.67093651 -0.67086298 -0.67106051 -0.6707569  -0.6709012  -0.67079469
 -0.67107026 -0.67078829 -0.6708888  -0.67090774 -0.67098314 -0.67085242
 -0.67090045 -0.67090185 -0.6708172  -0.67095829 -0.67072842 -0.67104207
 -0.67083438 -0.67075394 -0.671175   -0.670783   -0.67103044 -0.67095615
 -0.67106148 -0.67076434 -0.67084    -0.67071577 -0.67084998 -0.67076546
 -0.67087354 -0.67091272 -0.67085882 -0.67067797 -0.67070526 -0.67094305
 -0.67086019 -0.67081095 -0.67075315 -0.67081706 -0.67074611 -0.67102152
 -0.67107574 -0.67080381 -0.67070105]
-0.6706911610691463
rewards2
[-0.67054525 -0.67065657 -0.67081871 -0.67077826 -0.67091624 -0.67052008
 -0.67071004 -0.67072301 -0.67088015 -0.67067644 -0.67064789 -0.67074992
 -0.67068531 -0.67073783 -0.67060186 -0.67074926

-0.6689242181387973
rewards12
[-0.66895017 -0.66885992 -0.66882149 -0.66890942 -0.66886418 -0.66875842
 -0.66883676 -0.66901484 -0.66889618 -0.66885755 -0.66887827 -0.66893405
 -0.66895848 -0.66894261 -0.6689124  -0.66894475 -0.66900333 -0.6687021
 -0.66884329 -0.66889822 -0.66875179 -0.66906811 -0.66864104 -0.66895462
 -0.6689725  -0.66874874 -0.66892099 -0.66904643 -0.66900949 -0.66891246
 -0.66918017 -0.66880613 -0.66891715 -0.66891423 -0.66893488 -0.66912445
 -0.66905521 -0.66893521 -0.66889058 -0.66891668 -0.66896457 -0.66888164
 -0.6691148  -0.66893961 -0.66895976 -0.66901831 -0.66874711 -0.66897904
 -0.66877493 -0.66892174 -0.66903521 -0.66898815 -0.66899214 -0.66891213
 -0.66902654 -0.66893628 -0.66882995 -0.66902319 -0.66913765 -0.66891102
 -0.66893087 -0.66902939 -0.66897968]
-0.6687250837596148
rewards13
[-0.66872155 -0.66854073 -0.66848645 -0.66859351 -0.66890718 -0.6688327
 -0.66877985 -0.6688279  -0.66879779 -0.66845962 -0.66865137 -0.66863164
 -0.66885529 -0.66871811 -0.

In [ ]:
# load the weights from file
# Not working know


#state = env.reset()
env = env_pybullet_kin_gen3()
env.update_parameters_to_modify(["mass","max_vel","kp","ki","damping","force_x_one","Ixx","Iyy","Izz"])
env.robot.visual_inspection = False
env.modified_parameters_df = env.create_df_from_Excel("./Parameters_train_tcp_euc.xlsx")


t.sleep(0.02)
action = env.action_modified()
action = np.array(action)
print(action)
reward = env.step_tcp_rishabh(action)

print("reward")
print(reward)
    

In [ ]:
#Convert to excel
a = env.df_avg.to_numpy()
print(a[:,5])
env.df_avg.to_excel("./Train_parameters_result_tcp_euc.xlsx")

In [ ]:
env.original_parameters_df